In [37]:
# 모든 작업에 앞서 이 코드를 실행시켜주기 바람
from bs4 import BeautifulSoup
import urllib.request
import re

def crawler(category_num):
    #웹브라우져 헤더 추가
    #User-Agent를 조작하는 경우
    hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    req = urllib.request.Request('https://prod.danawa.com/list/?cate='+category_num, headers = hdr)
    data = urllib.request.urlopen(req).read()
    page = data.decode('utf-8','ignore')
    soup = BeautifulSoup(page, 'html.parser')
    prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
    return prod_items

In [38]:
def cpu_crawler(link):
    prod_items = crawler(link)
    i=0
    partslist = []
    for item in prod_items:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box--full').get_text()
        details = re.split("[/'\n']", details) # 상세정보들
        if link[-6:] == '113973': # intel 소캣 split
            socket = details[0].strip()[5:-1] if details else 'N/A' 
        elif link[-6:] == '113990': # amd 소켓 splite
            socket = details[0].strip()[6:-1] if details else 'N/A'
        integrated_graphics = True if '내장그래픽: 탑재' in details else False # 내장그래픽
        for detail in details:
            if '메모리 규격' in detail:
                memory_type = str(detail[8:])
            if '시네벤치R23(싱글)' in detail:
                singlebench = detail[13:]
            if '시네벤치R23(멀티)' in detail:
                multibench = detail[13:]
        partslist.append([name, price, socket, integrated_graphics, memory_type, singlebench, multibench])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Socket', 'Integrated Graphics','메모리 타입','시네벤치R23(싱글)','시네벤치R23(멀티)'])
    return result

intel_link ='113973'
amd_link = '113990'
intel_cpu = cpu_crawler(intel_link)
amd_cpu = cpu_crawler(amd_link)

In [39]:
amd_cpu.tail()

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
25,AMD 라이젠9-4세대 5900X (버미어),"455,900",,False,DDR4,1578,20811
26,AMD 라이젠5-5세대 7600X (라파엘),"301,210",,True,DDR5,1961,15215
27,AMD 라이젠7-4세대 5700G (세잔),"340,330",,True,DDR4,1507,13898
28,AMD 라이젠7-5세대 7700X (라파엘),"510,000",,True,DDR5,2018,20059
29,AMD 라이젠7-4세대 5800X (버미어),"631,750",,False,DDR4,1593,15416


In [40]:
intel_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,인텔 코어i5-14세대 14400F (랩터레이크 리프레시),"281,000",,False,"DDR5, DDR4",1790,17804
1,인텔 코어i7-14세대 14700K (랩터레이크 리프레시),"581,030",,True,"DDR5, DDR4",2069,34818
2,인텔 코어i5-12세대 12400F (엘더레이크),"152,340",,False,"DDR5, DDR4",1605,12201
3,인텔 코어i5-14세대 14500 (랩터레이크 리프레시),"345,400",,True,"DDR5, DDR4",1934,22045
4,인텔 코어i9-14세대 14900K (랩터레이크 리프레시),"812,710",,True,"DDR5, DDR4",2277,40146
5,인텔 코어i7-14세대 14700F (랩터레이크 리프레시),"513,810",,False,"DDR5, DDR4",2091,32090
6,인텔 코어i7-13세대 13700KF (랩터레이크),"464,980",,False,"DDR5, DDR4",1978,30354
7,인텔 코어i5-13세대 13400F (랩터레이크),"237,000",,False,"DDR5, DDR4",1789,14621
8,인텔 코어i7-14세대 14700KF (랩터레이크 리프레시),"545,930",,False,"DDR5, DDR4",2145,35218
9,인텔 코어i3-12세대 12100F (엘더레이크),"105,290",,False,"DDR5, DDR4",1666,8376


In [41]:
intel_cpu.to_csv('intel.csv',encoding='cp949', index=False)
intel_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,인텔 코어i5-14세대 14400F (랩터레이크 리프레시),"281,000",,False,"DDR5, DDR4",1790,17804
1,인텔 코어i7-14세대 14700K (랩터레이크 리프레시),"581,030",,True,"DDR5, DDR4",2069,34818
2,인텔 코어i5-12세대 12400F (엘더레이크),"152,340",,False,"DDR5, DDR4",1605,12201
3,인텔 코어i5-14세대 14500 (랩터레이크 리프레시),"345,400",,True,"DDR5, DDR4",1934,22045
4,인텔 코어i9-14세대 14900K (랩터레이크 리프레시),"812,710",,True,"DDR5, DDR4",2277,40146
5,인텔 코어i7-14세대 14700F (랩터레이크 리프레시),"513,810",,False,"DDR5, DDR4",2091,32090
6,인텔 코어i7-13세대 13700KF (랩터레이크),"464,980",,False,"DDR5, DDR4",1978,30354
7,인텔 코어i5-13세대 13400F (랩터레이크),"237,000",,False,"DDR5, DDR4",1789,14621
8,인텔 코어i7-14세대 14700KF (랩터레이크 리프레시),"545,930",,False,"DDR5, DDR4",2145,35218
9,인텔 코어i3-12세대 12100F (엘더레이크),"105,290",,False,"DDR5, DDR4",1666,8376


In [42]:
amd_cpu.to_csv('amd.csv',encoding='cp949', index=False)
amd_cpu

,Product Name,Price,Socket,Integrated Graphics,메모리 타입,시네벤치R23(싱글),시네벤치R23(멀티)
0,AMD 라이젠5-5세대 7500F (라파엘),"194,700",,False,DDR5,1824,13824
1,AMD 라이젠7-5세대 7800X3D (라파엘),"527,030",,True,DDR5,1788,18208
2,AMD 라이젠5-4세대 5600 (버미어),"154,000",,False,DDR4,1460,10906
3,AMD 라이젠5-5세대 8600G (피닉스),"262,380",,True,DDR5,1662,13513
4,AMD 라이젠5 PRO 4650G (르누아르),"101,160",,True,DDR4,1662,13513
5,AMD 라이젠5-4세대 5600G (세잔),"280,000",,True,DDR4,1446,10489
6,AMD 라이젠7-4세대 5700X3D (버미어),"283,300",,False,DDR4,1446,10489
7,AMD 라이젠5-5세대 8500G (피닉스),"202,280",,True,DDR5,1807,11541
8,AMD 라이젠9-5세대 7950X3D (라파엘),"833,300",,True,DDR5,2060,36372
9,AMD 라이젠5-5세대 7600 (라파엘),"253,990",,True,DDR5,1864,14387


In [43]:
# 램 용량이 크롤링 되지 않아 수정하였음
ddr4_link = '1131326'
ddr5_link = '11341201'

def memory_crawling(link):
    crolled_data = crawler(link)
    i=0
    partslist = list()
    for item in crolled_data:
        if i==30: break;
        storage = item.select('div.over_preview > p.memory_sect > span.text')
        price = item.select('p > a > strong')
        for j in range(len(storage)):
            name = item.select_one('p > a').get_text()[8:].strip() # 부품명 # strip 메서드로 앞 뒤 공백 제거
            details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
            speed = details[2] # 메모리 클럭
            price_ = price[j].get_text()
            storage_ = storage[j].get_text()
            if(len(storage_)) == 3: # 8GB나 4GB인 애들은 길이가 3이니까
                storage_ = str(int(storage_[:1])*2)+'GB('+storage_[:1]+'Gx2)'
            elif len(storage_) == 4: # 16GB~64GB까지
                storage_ = str(int(storage_[:2])*2)+'GB('+storage_[:2]+'Gx2)'
            elif len(storage_) == 5: # 128GB~512GB 까지
                storage_ = str(int(storage_[:3])*2)+'GB('+storage_[:3]+'Gx2)'
            partslist.append([name, price_, storage_, speed])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price','Storage', 'Speed'])
    return result
ddr4 = memory_crawling(ddr4_link)
ddr5 = memory_crawling(ddr5_link)
ddr5

,Product Name,Price,Storage,Speed
0,마이크론 Crucial DDR5-5600 CL46 PRO 패키지 대원씨티에스,"368,900",96GB(48Gx2),5600MHz (PC5-44800)
1,마이크론 Crucial DDR5-5600 CL46 PRO 패키지 대원씨티에스,"262,870",64GB(32Gx2),5600MHz (PC5-44800)
2,마이크론 Crucial DDR5-5600 CL46 PRO 패키지 대원씨티에스,"176,760",48GB(24Gx2),5600MHz (PC5-44800)
3,마이크론 Crucial DDR5-5600 CL46 PRO 패키지 대원씨티에스,"117,370",32GB(16Gx2),5600MHz (PC5-44800)
4,삼성전자 DDR5-5600,"130,790",64GB(32Gx2),5600MHz (PC5-44800)
...,...,...,...,...
57,ESSENCORE KLEVV DDR5-8200 CL38 CRAS V RGB 패키지 서린,"459,990",48GB(24Gx2),8200MHz (PC5-65600)
58,PNY XLR8 DDR5-6400 CL32 MAKO RGB 블랙 패키지 마이크로닉스,"167,910",32GB(16Gx2),6400MHz (PC5-51200)
59,G.SKILL DDR5-6400 CL32 RIPJAWS M5 RGB 블랙 패키지,"548,990",96GB(48Gx2),6400MHz (PC5-51200)
60,G.SKILL DDR5-6400 CL32 RIPJAWS M5 RGB 블랙 패키지,"348,000",64GB(32Gx2),6400MHz (PC5-51200)


In [44]:
# 램 가격 대상으로만 크롤링 - 사실상 초기 버전
ddr4_link = '1131326'
ddr5_link = '11341201'

def memory_crawling(link):
    crolled_data = crawler(link)
    # 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
    i=0
    partslist = list()
    for item in crolled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        speed = details[2]
        partslist.append([name, price, speed])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price','Speed'])
    return result
ddr4 = memory_crawling(ddr4_link)
ddr5 = memory_crawling(ddr5_link)
ddr4.to_csv('ddr4.csv',encoding='cp949', index=False)
ddr5.to_csv('ddr5.csv',encoding='cp949', index=False)
ddr4

,Product Name,Price,Speed
0,ESSENCORE KLEVV DDR4-3200 CL22 파인인포,"73,310",3200MHz (PC4-25600)
1,삼성전자 DDR4-3200,"92,230",3200MHz (PC4-25600)
2,삼성전자 DDR4-2666,"42,880",2666MHz (PC4-21300)
3,마이크론 Crucial DDR4-3200 CL22 대원씨티에스,"41,860",3200MHz (PC4-25600)
4,ESSENCORE KLEVV DDR4-3600 CL18 BOLT XR 패키지 서린,"100,990",3600MHz (PC4-28800)
5,TeamGroup T-Force DDR4-3600 CL18 Delta RGB 화이트 패키지 서린,"240,000",3600MHz (PC4-28800)
6,G.SKILL DDR4-3600 CL18 TRIDENT Z RGB 패키지,"221,930",3600MHz (PC4-28800)
7,마이크론 Crucial DDR4-3200 CL22,"82,440",3200MHz (PC4-25600)
8,ESSENCORE KLEVV DDR4-3200 CL16 BOLT X 서린,"44,890",3200MHz (PC4-25600)
9,GeIL DDR4-2666 CL19 PRISTINE,"37,990",2666MHz (PC4-21300)


In [47]:
# 메인보드 크롤링
import re
import pandas as pd
def board_crawling(link):
    crolled_data = crawler(link)
    i=0
    partslist = list()
    for item in crolled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        socket_type = re.split("[()]", details[0])[1][2:] # details 중 0 번째에 위치하고 있으며, AMD(소캣AM5) 이런식으로 되어 있어서 쪼갰음
        # print(socket_type)
        for idx in range(len(details)):
            if '메모리' in details[idx]:
                memory_type = str(details[idx][4:])
                memory_speed = str(details[idx+1].split()[0])
                break
        size = details[2].split()[0]
        partslist.append([name, price, socket_type, memory_type, memory_speed, size])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Socket_type', 'Memory_Type','Memory_Speed', 'Size'])
    return result
amd_board_link = '1131249'
intel_board_link = '11345282'
amd_board = board_crawling(amd_board_link)
intel_board = board_crawling(intel_board_link)
amd_board

,Product Name,Price,Socket_type,Memory_Type,Memory_Speed,Size
0,ASRock B650M PG Lightning 에즈윈,"159,030",AM5,DDR5,7200MHz,M-ATX
1,MSI MAG B650M 박격포 WIFI,"247,690",AM5,DDR5,7600MHz,M-ATX
2,ASUS PRIME A520M-A II 대원씨티에스,"81,500",AM4,DDR4,4866MHz,M-ATX
3,ASUS TUF Gaming B550M-PLUS STCOM,"155,170",AM4,DDR4,4800MHz,M-ATX
4,ASRock B650M PG Lightning 대원씨티에스,"149,580",AM5,DDR5,7200MHz,M-ATX
5,MSI MAG B650 토마호크 WIFI,"290,550",AM5,DDR5,7600MHz,ATX
6,GIGABYTE B650M K 제이씨현,"164,800",AM5,DDR5,8000MHz,M-ATX
7,GIGABYTE B650M K 피씨디렉트,"159,480",AM5,DDR5,8000MHz,M-ATX
8,ASUS PRIME A520M-K 대원씨티에스,"68,600",AM4,DDR4,4600MHz,M-ATX
9,GIGABYTE X670E AORUS PRO X 피씨디렉트,"408,250",AM5,DDR5,8000MHz,ATX


In [48]:
intel_board

,Product Name,Price,Socket_type,Memory_Type,Memory_Speed,Size
0,MSI MAG B760M 박격포 맥스 WIFI,"212,690",1700,DDR5,7000MHz,M-ATX
1,ASUS PRIME H610M-K D4 인텍앤컴퍼니,"97,500",1700,DDR4,3200MHz,M-ATX
2,ASUS TUF Gaming B760M-PLUS II 코잇,"197,890",1700,DDR5,7800MHz,M-ATX
3,GIGABYTE B760M AORUS ELITE 피씨디렉트,"188,660",1700,DDR5,7800MHz,M-ATX
4,ASUS ROG STRIX B760-G GAMING WIFI 코잇,"231,460",1700,DDR5,7800MHz,M-ATX
5,MSI PRO B760M-A WIFI,"167,710",1700,DDR5,6800MHz,M-ATX
6,GIGABYTE B760M DS3H D4 피씨디렉트,"137,970",1700,DDR4,5333MHz,M-ATX
7,MSI PRO B760M-A DDR4 II,"147,690",1700,DDR4,5333MHz,M-ATX
8,MSI MAG B760M 박격포 II,"193,730",1700,DDR5,7800MHz,M-ATX
9,GIGABYTE B760M E 피씨디렉트,"126,990",1700,DDR5,8000MHz,M-ATX


In [49]:
amd_board.to_csv('amd_board.csv',encoding='cp949', index=False)
intel_board.to_csv('intel_board.csv',encoding='cp949', index=False)

In [50]:
# GPU 크롤링
def gpu_crawling(link):
    crawled_data = crawler(link)
    i=0
    partslist = list()
    for item in crawled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        for idx in range(len(details)):
            if '정격파워' in details[idx]:
                # power_usage = str(details[idx].split()[1])
                power_type = str(details[idx].split()[1])
            if '가로(길이)' in details[idx]:
                size = str(details[idx].split()[1])
                break
        partslist.append([name, price, power_type, size])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'Power_Type', '가로(길이)'])
    return result
gpu_link = '112753'
gpu = gpu_crawling(gpu_link)
gpu.to_csv('gpu.csv',encoding='cp949', index=False)
gpu

,Product Name,Price,Power_Type,가로(길이)
0,이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB,"417,530",600W,249.9mm
1,이엠텍 지포스 RTX 4060 Ti STORM X Dual OC D6 8GB,"556,930",650W,249.9mm
2,갤럭시 GALAX 지포스 RTX 4060 WHITE OC D6 8GB,"429,730",550W,237mm
3,XFX 라데온 RX 6800 SWFT 319 CORE D6 16GB,"500,320",650W,340mm
4,MSI 지포스 RTX 4070 SUPER 게이밍 X 슬림 D6X 12GB 트라이프로져3,"974,700",650W,307mm
5,ZOTAC GAMING 지포스 RTX 4060 TWIN Edge OC D6 8GB,"415,310",500W,225.3mm
6,MSI 지포스 RTX 4070 Ti SUPER 벤투스 3X OC D6X 16GB,"1,158,440",700W,308mm
7,MSI 지포스 RTX 4060 Ti 벤투스 2X 블랙 OC D6 16GB,"615,670",550W,199mm
8,MSI 지포스 RTX 3060 벤투스 2X OC D6 12GB,"393,670",550W,235mm
9,이엠텍 지포스 RTX 4060 MIRACLE WHITE D6 8GB,"426,480",600W,252mm


In [52]:
!pip install selenium
# 다른 프로젝트(자연어 처리) 에 사용되는 torch 패키지랑 충돌나네.... 젠장!

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB 9.2 MB/s eta 0:00:01
   -- ------------------------------------- 0.7/9.4 MB 8.9 MB/s eta 0:00:01
   ---- ----------------------------------- 1.0/9.4 MB 7.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.5/9.4 MB 8.6 MB/s eta 0:00:01
   ------ --------------------------------- 1.5/9.4 MB 8.7 MB/s eta 0:00:01
   --------- ------------------------------ 2.3/9.4 MB 8.7 MB/s eta 0:00:01
   ----------- ---------------------------- 2.7/9.4 MB 8.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.1/9.4 MB 8.6 MB/s eta 0:00:01
   --------------- ------------------------ 3.6/9.4 MB 8.8 MB/s eta 0:00:01
   ----------------- ---------------------- 4.0/9.4 MB 8.9 MB/s eta 0:00:01
   ------------------- -------------------- 4.5/9.4 MB 9.3 MB/s eta 0:00:01
   --------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, but you have pyqt5 5.15.10 which is incompatible.
spyder 5.2.2 requires pyqtwebengine<5.13, but you have pyqtwebengine 5.15.6 which is incompatible.


In [54]:
# 파워 크롤링의 다른 방식
# why? 페이지 5까지 크롤링 가능해서 최대 150개까지 크롤링 가능하기 때문

import time
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

# 브라우저 띄우지 않고 하기
options = ChromeOptions()
options.add_argument('headless')

url = 'https://prod.danawa.com/list/?cate=112777'
driver = Chrome()
driver.get(url)

time.sleep(2)

j=0
partslist = list() # make empty list

for page in range(1,6):# 5page
    soup = BeautifulSoup(driver.page_source)
    product_li_tags = soup.select('li.prod_item.prod_layer')
    for li in product_li_tags:
        name = li.select_one('p.prod_name a').text.strip()
        price = li.select_one('p.price_sect a strong').text.strip()
        details = li.select_one('div.spec_list').get_text().split('/')
        for idx in range(len(details)):
            if '파워' in details[idx]:
                power_size = str(details[idx].split()[0])
            if '정격출력' in details[idx]:
                output = str(details[idx].split()[1])
                break
        img_link = li.select_one('div.thumb_image img').get('data-original')
        if img_link == None:
            img_link = li.select_one('div.thumb_image img').get('src')[2:] # img link가 //으로 들어와서 //없에버림
        # print(j,name, price, power_size, output, img_link)
        partslist.append([name, price, power_size, output, img_link])
        j+=1
power = pd.DataFrame(partslist, columns=['Product Name', 'Price', '파워규격', '정격출력','img_link'])
power.to_csv('power.csv',encoding='cp949', index=False)
power

,Product Name,Price,파워규격,정격출력,img_link
0,앱코 SETTLER 하이브리드 PCIE5.1 STH-700B ETA BRONZE 화이트,"65,000",ATX,700W,img.danawa.com/prod_img/500000/012/742/img/49742012_1.jpg?shrink=130:130&_v=20240726174519
1,마이크로닉스 Classic II 풀체인지 700W 80PLUS브론즈 ATX3.1,"83,900",ATX,700W,img.danawa.com/prod_img/500000/700/642/img/49642700_1.jpg?shrink=130:130&_v=20240703124116
2,마이크로닉스 Classic II 풀체인지 600W 80PLUS브론즈 ATX3.1,"62,500",ATX,600W,img.danawa.com/prod_img/500000/652/642/img/49642652_1.jpg?shrink=130:130&_v=20240603170345
3,마이크로닉스 Classic II 850W 80PLUS골드 풀모듈러 ATX3.1,"169,000",ATX,850W,img.danawa.com/prod_img/500000/078/815/img/51815078_1.jpg?shrink=130:130&_v=20240507152526
4,시소닉 VERTEX GX-850 GOLD 풀모듈러 ATX3.0,"208,000",ATX,850W,//img.danawa.com/prod_img/500000/063/698/img/18698063_1.jpg?shrink=130:130&_v=20240528142232
...,...,...,...,...,...
150,맥스엘리트 MAXWELL BARON 800W 80PLUS브론즈 플랫,"87,000",ATX,800W,//img.danawa.com/prod_img/500000/560/200/img/16200560_1.jpg?shrink=130:130&_v=20240508102014
151,마이크로닉스 WIZMAX 1200W 80PLUS플래티넘 풀모듈러 ATX3.1,"299,000",ATX,1200W,//img.danawa.com/prod_img/500000/218/223/img/35223218_1.jpg?shrink=130:130&_v=20240619081351
152,쿨러마스터 MWE 700 BRONZE V2 230V,"71,800",ATX,700W,//img.danawa.com/prod_img/500000/805/400/img/12400805_1.jpg?shrink=130:130&_v=20210107133641
153,마이크로닉스 Compact SFX 700W 80PLUS골드,"139,000",M-ATX(SFX),700W,//img.danawa.com/prod_img/500000/575/759/img/10759575_1.jpg?shrink=130:130&_v=20240507153840


In [17]:
# 파워 크롤링 - 원래 방식

def power_crawling(link):
    crawled_data = crawler(link)
    i=0
    partslist = list()
    for item in crawled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        # link = 
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        for idx in range(len(details)):
            if '파워' in details[idx]:
                power_size = str(details[idx].split()[0])
            if '정격출력' in details[idx]:
                output = str(details[idx].split()[1])
                break
        partslist.append([name, price, power_size, output])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', '파워규격', '정격출력'])
    return result

power_link = '11338807' # 80 PLUS 인증 파워만 가져옴
power = power_crawling(power_link)
power.to_csv('power.csv',encoding='cp949', index=False)
power

,Product Name,Price,파워규격,정격출력
0,마이크로닉스 Classic II 풀체인지 700W 80PLUS브론즈 ATX3.1,"78,031",ATX,700W
1,마이크로닉스 Classic II 풀체인지 600W 80PLUS브론즈 ATX3.1,"62,500",ATX,600W
2,잘만 GigaMax III 850W 80PLUS브론즈 모듈러 ATX3.1,"90,540",ATX,850W
3,마이크로닉스 Classic II 풀체인지 500W 80PLUS브론즈 ATX3.1,"54,500",ATX,500W
4,마이크로닉스 Classic II 850W 80PLUS골드 풀모듈러 ATX3.1,"159,008",ATX,850W
5,마이크로닉스 Classic II 풀체인지 800W 80PLUS브론즈 ATX3.1,"92,915",ATX,800W
6,잘만 MegaMax 600W 80PLUS스탠다드,"50,650",ATX,600W
7,시소닉 VERTEX GX-1000 GOLD 풀모듈러 ATX3.0,"254,000",ATX,1000W
8,마이크로닉스 Classic II 풀체인지 700W 80PLUS브론즈 ATX3.1 화이트,"86,650",ATX,700W
9,마이크로닉스 Classic II 1050W 80PLUS골드 230V EU 풀모듈러 화이트,"185,000",ATX,1050W


In [ ]:
# 케이스 크롤링 - 원래 방식

def case_crawling(link):
    crawled_data = crawler(link)
    i=0
    partslist = list()
    for item in crawled_data:
        if i==30: break;
        # strip 메서드로 앞 뒤 공백 제거
        name = item.select_one('p > a').get_text()[8:].strip() # 부품명
        price = item.select_one('p > a > strong').get_text() # 가격
        # link = 
        details = item.select_one('div.spec-box.spec-box--full > div.spec_list').get_text().split('/') # 상세정보들
        for idx in range(len(details)):
            if 'VGA 장착 길이' in details[idx]:
                vga_length = str(details[idx].split()[3])
            if 'CPU쿨러 장착높이' in details[idx]:
                cpuCooler_height = str(details[idx].split()[2])
                break
        partslist.append([name, price, vga_length, cpuCooler_height])
        i+=1
    result = pd.DataFrame(partslist, columns=['Product Name', 'Price', 'VGA 장착길이', 'CPU쿨러 장착높이'])
    return result

case_link = '113971'
case = case_crawling(case_link)
case

In [19]:
# 다나와 인텔 CPU 가격만 크롤링 코드 - 초기 버전
from bs4 import BeautifulSoup
import urllib.request
import re

#User-Agent를 조작하는 경우(아이폰에서 사용하는 사파리 브라우져의 헤더)
hdr =  {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
data ='https://prod.danawa.com/list/?cate=113973'
#웹브라우져 헤더 추가
req = urllib.request.Request(data, \
                            headers = hdr)

data = urllib.request.urlopen(req).read()
page = data.decode('utf-8','ignore')
soup = BeautifulSoup(page, 'html.parser')
crolled_data = soup.find_all(['p','strong'],attrs={'class':'price_sect'})

# 다나와 특성상 상위 4개 및 9번째 위치에 광고를 노출함, 광고를 제거하기 위해 i 변수 사용
i=0
partslist = list()
for item in crolled_data:
    if i>=4 and i!=9:
        # strip 메서드로 앞 뒤 공백 제거
        print(item.select_one('strong').get_text())
    i+=1
print(partslist)


283,080
281,740
271,340
312,410
583,640
535,990
152,030
134,624
123,360
150,000
347,270
348,900
338,260
398,750
814,470
791,290
659,040
152,030
238,150
227,090
217,660
247,350
516,300
500,000
507,620
601,170
547,920
506,370
607,890
404,480
331,190
342,990
328,940
538,290
104,850
96,352
86,910
104,740
462,500
454,420
444,990
510,640
321,820
285,730
271,740
379,180
445,390
424,070
481,380
390,920
396,510
384,870
419,810
162,710
162,550
152,820
210,100
969,370
563,860
564,300
539,000
700,550
358,990
333,160
322,150
363,910
155,410
111,970
129,210
149,740
295,300
259,990
241,800
243,000
415,690
154,370
123,260
118,140
100,150
402,050
422,600
272,920
268,960
199,690
179,000
167,450
266,350
404,430
368,690
417,550
773,680
763,140
899,360
208,440
182,050
194,560
257,890
352,790
303,900
291,000
548,700
377,920
318,070
334,530
434,740
233,630
206,290
207,320
228,770
269,390
274,590
261,640
271,650
647093
[]
